# 🚀 KAN-PINN JAX GPU Optimization - Levitador Magnético## Identificación de Parámetros Físicos con Differential Evolution en GPUEste notebook implementa un sistema completo de optimización para identificar los parámetros físicos del levitador magnético usando:- **JAX**: Computación vectorizada en GPU- **Differential Evolution**: Optimización metaheurística completamente paralelizada- **GPU Acceleration**: Evaluación de toda la población en un solo paso### 📋 Contenido del Notebook:1. **Configuración del Entorno** (Colab GPU, dependencias)2. **Carga de Datos** (formato estándar y KAN-PINN)3. **Transferencia a GPU** (pandas → JAX arrays)4. **Modelo Físico** (vectorizado en JAX)5. **Differential Evolution** (optimización GPU)6. **Visualización de Resultados**7. **Guardado y Descarga**8. **Comparativa GPU vs CPU** (opcional)### ⚡ Instrucciones de Uso:1. **Activar GPU en Colab**: `Runtime > Change runtime type > GPU (T4)`2. **Ejecutar todas las celdas**: `Runtime > Run all`3. El notebook se auto-configura y usa datos demo del repositorio---

## 1️⃣ Configuración del EntornoInstalamos dependencias y verificamos que tenemos acceso a GPU.

In [ ]:
# Verificar GPU disponibleimport subprocessimport sysprint("🔍 Verificando GPU...")try:    gpu_info = subprocess.check_output(['nvidia-smi'], text=True)    print("✅ GPU detectada:")    print(gpu_info)except:    print("⚠️  No se detectó GPU. Este notebook funcionará en CPU (más lento).")    print("   Para activar GPU: Runtime > Change runtime type > GPU")

In [ ]:
# Instalar dependencias necesariasprint("📦 Instalando dependencias...\n")# JAX con soporte CUDA!pip install -q "jax[cuda12]>=0.4.20" jaxlib>=0.4.20# Otras librerías necesarias!pip install -q numpy scipy pandas matplotlib tqdmprint("\n✅ Dependencias instaladas correctamente")

In [ ]:
# Verificar instalación de JAX y dispositivos disponiblesimport jaximport jax.numpy as jnpfrom jax import random, jit, vmapfrom jax.lax import scanimport numpy as npimport pandas as pdimport matplotlib.pyplot as pltfrom pathlib import Pathimport jsonimport timefrom datetime import datetimefrom tqdm.auto import tqdmfrom functools import partial# Verificar dispositivos JAXprint("\n🖥️  Dispositivos JAX disponibles:")devices = jax.devices()for i, device in enumerate(devices):    print(f"  [{i}] {device}")# Forzar uso de GPU si está disponibleif any('gpu' in str(d).lower() for d in devices):    print("\n✅ GPU disponible - Los cálculos se ejecutarán en GPU")else:    print("\n⚠️  GPU no disponible - Los cálculos se ejecutarán en CPU")# Configurar matplotlibplt.style.use('seaborn-v0_8-darkgrid')%matplotlib inlineprint("\n✅ Imports completados correctamente")

## 2️⃣ Clonar Repositorio (Opcional)Si estamos en Colab y queremos usar los datos del repositorio, clonamos el repo.

In [ ]:
# Verificar si ya estamos en el repo o necesitamos clonarloimport osREPO_URL = "https://github.com/JRavenelco/levitador-benchmark.git"REPO_DIR = "levitador-benchmark"if os.path.exists(REPO_DIR):    print(f"✅ Repositorio ya existe en {REPO_DIR}")    %cd {REPO_DIR}elif os.path.exists("levitador_benchmark.py"):    print("✅ Ya estamos en el directorio del repositorio")    REPO_DIR = "."else:    print(f"📥 Clonando repositorio desde {REPO_URL}...")    !git clone {REPO_URL}    %cd {REPO_DIR}    print("✅ Repositorio clonado correctamente")# Mostrar estructura de datos disponiblesprint("\n📁 Archivos de datos disponibles:")if os.path.exists("data"):    !ls -lh data/*.txt 2>/dev/null || echo "  (No hay archivos .txt en data/)"    if os.path.exists("data/sesiones_kan_pinn"):        print("\n📁 Datos KAN-PINN disponibles:")        !ls -lh data/sesiones_kan_pinn/*.txt 2>/dev/null || echo "  (No hay archivos)"else:    print("  ⚠️  Carpeta 'data' no encontrada")

## 3️⃣ Funciones de Carga de DatosFunciones flexibles para cargar datos en diferentes formatos:- **Formato estándar**: `datos_levitador.txt` (columnas: t, y, i, u, ...)- **Formato KAN-PINN**: `sesiones_kan_pinn/dataset_*.txt` (columnas: t, y, y_obs, dy_obs, i, u, yd)

In [ ]:
def load_standard_data(filepath, subsample=1):    """    Carga datos en formato estándar del levitador.        Formato esperado (columnas separadas por tabs/espacios):    t [s]  |  y [m]  |  i [A]  |  u [V]  |  dy [m/s]  |  yd [m]        Args:        filepath: Ruta al archivo de datos        subsample: Factor de submuestreo (1=todos, 10=cada 10 puntos)        Returns:        dict con arrays numpy: 't', 'y', 'i', 'u', 'dy' (opcional), 'yd' (opcional)    """    print(f"\n📂 Cargando datos desde: {filepath}")        # Detectar si tiene encabezado    with open(filepath, 'r') as f:        first_line = f.readline().strip()        skiprows = 1 if (first_line.startswith('#') or not first_line[0].isdigit()) else 0        # Cargar con pandas para manejo robusto    if skiprows > 0:        df = pd.read_csv(filepath, sep='\\s+', comment='#', header=None)    else:        df = pd.read_csv(filepath, sep='\\s+', header=None)        # Submuestreo    if subsample > 1:        df = df.iloc[::subsample].reset_index(drop=True)        print(f"   Submuestreo: {subsample}x (quedaron {len(df)} puntos)")        # Extraer columnas    data = {        't': df.iloc[:, 0].values.astype(np.float32),        'y': df.iloc[:, 1].values.astype(np.float32),        'i': df.iloc[:, 2].values.astype(np.float32),        'u': df.iloc[:, 3].values.astype(np.float32),    }        # Columnas opcionales    if df.shape[1] > 4:        data['dy'] = df.iloc[:, 4].values.astype(np.float32)    if df.shape[1] > 5:        data['yd'] = df.iloc[:, 5].values.astype(np.float32)        print(f"✅ Datos cargados: {len(data['t'])} muestras")    print(f"   Columnas: {list(data.keys())}")    print(f"   Rango temporal: {data['t'][0]:.3f} - {data['t'][-1]:.3f} s")        return datadef load_kanpinn_data(filepath, subsample=1):    """    Carga datos en formato KAN-PINN.        Formato esperado (con encabezado comentado):    # Columnas: t y y_obs dy_obs i u yd    t [s]  |  y [m]  |  y_obs [m]  |  dy_obs [m/s]  |  i [A]  |  u [V]  |  yd [m]        Args:        filepath: Ruta al archivo de datos KAN-PINN        subsample: Factor de submuestreo        Returns:        dict con arrays numpy: 't', 'y', 'y_obs', 'dy_obs', 'i', 'u', 'yd'    """    print(f"\n📂 Cargando datos KAN-PINN desde: {filepath}")        # Leer con pandas (robusto para diferentes encodings)    df = pd.read_csv(filepath, sep='\\s+', comment='#', header=None,                     encoding='utf-8', encoding_errors='ignore')        # Submuestreo    if subsample > 1:        df = df.iloc[::subsample].reset_index(drop=True)        print(f"   Submuestreo: {subsample}x (quedaron {len(df)} puntos)")        # Extraer columnas según formato KAN-PINN    data = {        't': df.iloc[:, 0].values.astype(np.float32),        'y': df.iloc[:, 1].values.astype(np.float32),        'y_obs': df.iloc[:, 2].values.astype(np.float32),        'dy_obs': df.iloc[:, 3].values.astype(np.float32),        'i': df.iloc[:, 4].values.astype(np.float32),        'u': df.iloc[:, 5].values.astype(np.float32),    }        if df.shape[1] > 6:        data['yd'] = df.iloc[:, 6].values.astype(np.float32)        print(f"✅ Datos KAN-PINN cargados: {len(data['t'])} muestras")    print(f"   Columnas: {list(data.keys())}")    print(f"   Rango temporal: {data['t'][0]:.3f} - {data['t'][-1]:.3f} s")        return datadef auto_load_data(filepath, subsample=1):    """    Carga automática detectando el formato.        Detecta si es formato KAN-PINN (más columnas, encabezado específico)    o formato estándar.    """    # Leer primera línea para detectar formato    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:        lines = [f.readline() for _ in range(10)]        # Buscar indicadores de formato KAN-PINN    is_kanpinn = any('y_obs' in line or 'dy_obs' in line or 'dataset_' in str(filepath)                      for line in lines)        if is_kanpinn:        return load_kanpinn_data(filepath, subsample)    else:        return load_standard_data(filepath, subsample)print("✅ Funciones de carga de datos definidas")